In [1]:
import sys
import datetime as dt
from pathlib import Path
from collections import OrderedDict
import math

import requests
import gtfstk as gt
from highcharts import Highchart 

DIR = Path('..')
sys.path.append(str(DIR))
from transit_reliability import *

%load_ext autoreload
%autoreload 2

TESTS_DIR = DIR/'tests'/'data'
DATA_DIR = DIR/'data'/'auckland'

In [2]:
#dates = ['20160513', '20160514', '20160515', '20160516', '20160517', '20160518', '20160519']
date = '20160519'

In [3]:
# Load GTFS feed to access scheduling data
#feed = gt.read_gtfs(DATA_DIR/'auckland_gtfs_20160511.zip', dist_units_in='km') # Good till 20160518
feed = gt.read_gtfs(DATA_DIR/'auckland_gtfs_20160519.zip', dist_units_in='km') # Good from 20160519


In [26]:
# Clean delays and combine them for the entire date range
delays = pd.read_csv(DATA_DIR/'delays_{0}.csv.gz'.format(date), 
  compression='gzip', dtype={'stop_id': str, 'stop_sequence': int})

# Concatenate delays and stop times
f = gt.get_stop_times(feed, date)
f = f.merge(delays, how='left')

# Write to file
path = DATA_DIR/'augmented_stop_times_{0}.csv.gz'.format(date)
f.to_csv(str(path), index=False, compression='gzip')

f.T


,0,1,2,3,4,5,6,7,8,9,...,334149,334150,334151,334152,334153,334154,334155,334156,334157,334158
trip_id,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,1000015308-20160512154122_v40.34,...,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34,8300032432-20160512154122_v40.34
arrival_time,07:00:00,07:02:10,07:02:20,07:02:30,07:02:40,07:03:03,07:03:28,07:03:38,07:03:46,07:04:02,...,12:48:15,12:49:28,12:50:41,12:51:54,12:53:07,12:54:20,12:55:30,12:56:00,13:25:00,13:30:00
departure_time,07:00:00,07:02:10,07:02:20,07:02:30,07:02:40,07:03:03,07:03:28,07:03:38,07:03:46,07:04:02,...,12:48:15,12:49:28,12:50:41,12:51:54,12:53:07,12:55:00,12:55:30,12:56:00,13:25:00,13:30:00
stop_id,6920,6800,6364,6366,6182,6156,6319,6306,6308,6313,...,8418,8420,8422,8424,8426,8428,8430,8432,2006,2010
stop_sequence,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
stop_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pickup_type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
drop_off_type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,NaN,NaN
shape_dist_traveled,0,1.1846,1.4745,1.74088,2.01381,2.28962,2.88646,3.13798,3.45216,3.65087,...,6.34519,6.7551,7.11309,7.32066,7.71901,8.00679,8.40636,8.62261,23.0881,24.2706
arrival_delay,NaN,NaN,NaN,NaN,NaN,200,NaN,NaN,NaN,376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1291,-852


In [12]:
a = f['delay'].count()
b = f['delay'].shape[0]
print(a, b, a/b)

182772 334159 0.546961177164


In [ ]:
# # Merge scheduled and actual stop times for study date
# g = pd.read_csv(DATA_DIR/'gtfsr_stop_times_{0}.csv.gzip'.format(date), 
#   compression='gzip', dtype={'stop_id': str})
# del g['route_id']

# st = st.merge(g, how='left').sort_values(['trip_id', 'stop_sequence'])


# print('study date is', date)
# print('num scheduled stop times =', st.shape[0])
# print('num arrival delays=', st['arrival_delay'].count())
# print('num departure delays=', st['departure_delay'].count())
# print('num delays=', st['delay'].count())




In [ ]:
# Compute trips stats for later
ts = gt.compute_trips_stats(feed)
# rs = gt.compute_routes_stats(feed, ts, date)
# rs = rs.merge(feed.routes[['route_id', 'route_long_name']])


# Tukey box plots of stop delays by route

In [53]:
def agg_tukey(group):
    s = dict()
    dcol = 'delay'
    for i in range(1, 4):
        col = 'delay_q{0}'.format(i)
        s[col] = group[dcol].dropna().quantile(i/4)
          
    iqr = s['delay_q3'] - s['delay_q1']
    s['iqr'] = iqr
    cond = group[dcol] <= s['delay_q3'] + 1.5*iqr
    s['delay_high'] = group.loc[cond, dcol].max()
    cond = group[dcol] >= s['delay_q1'] - 1.5*iqr
    s['delay_low'] = group.loc[cond, dcol].min()
    s['num_delays'] = group[dcol].count()
    s['num_realtime_runs'] = (group['trip_id'] + group['date']).nunique()
    s['num_unique_trip_ids'] = group['trip_id'].nunique()
    return pd.Series(s)

f = all_delays.copy()
f = f.groupby('route_id').apply(agg_tukey).reset_index()
f = f.merge(feed.routes)
f.head()

# Only keep routes with an average of at least k delay samples per sample trip
k = 3
cond = f['num_delays']/f['num_realtime_runs'] >= k
g = f[cond].copy()

# Convert to minutes
box_cols = ['delay_low', 'delay_q1', 'delay_q2', 'delay_q3', 'delay_high']
g[box_cols] /= 60
g['route_name'] = g[['route_short_name', 'route_long_name']].apply(
  lambda x: ', '.join(x), axis=1) 

# Print 10 biggest IQR routes
print('Routes with largest IQR:')
print(g.sort_values('iqr', ascending=False).head(5))

# Plot
chart = Highchart()
data = g[['route_name'] + box_cols].copy()
chart.add_data_set(data.values.tolist(), series_type='boxplot')
N = data.shape[0]
date_range = ', '.join(dates)        
    
options = {
    'chart': {
        'height': 15*N,
        'inverted': True,
    },
    'plotOptions': {
    },
    'title': {
        'text': 'Tukey box plots of stop delays by route',
    },
    'subtitle': {
        'text': 'Sample period is {0}'.format(date_range)
    },
    'legend': {
        'enabled': False,
    },
    'xAxis': {
        'title': {
            'text': 'Route'
        },
        'type': 'category',
        'labels': {'step': 1},
    },
    'yAxis': {
        'title': {
            'text': 'Minutes'
        },
        'opposite': True,
        'plotLines': [{
            'value': 0,
            'color': 'red',
            'width': 2,
        }]    
    },
    'tooltip': {
#         'formatter': 'function () {\
#             return this.point.q3 - this.point.q1 + " min"\
#         }',
        'valueDecimals': 1,
        'headerFormat': '<b>{point.key}</b><br/>',
        'pointFormat': 'low: {point.low} min<br/>'\
          'Q1: {point.q1} min<br/>'\
          'Q2: {point.median} min<br/>'\
          'Q3: {point.q3} min<br/>'\
          'high: {point.high} min',
    }
}
chart.set_dict_options(options)
chart

Routes with largest IQR:
                        route_id  delay_high  delay_low   delay_q1  delay_q2  \
45   13011-20160415132758_v40.14   29.416667 -32.766667 -14.075000 -1.416667   
199  76003-20160512154122_v40.34   31.416667 -32.266667 -16.000000 -0.766667   
249  95202-20160512154122_v40.34   17.166667 -26.950000 -14.050000 -3.283333   
219  85003-20160415132758_v40.14   27.483333 -26.000000 -12.191667 -4.916667   
221  86003-20160512154122_v40.34   28.533333 -21.766667  -9.750000 -0.900000   

      delay_q3      iqr  num_delays  num_realtime_runs  num_unique_trip_ids  \
45   14.020833  1685.75       434.0               25.0                 10.0   
199   7.833333  1430.00      1301.0               60.0                 24.0   
249   3.566667  1057.00       379.0               20.0                  8.0   
219   4.658333  1011.00      1543.0               69.0                 28.0   
221   5.666667   925.00      1217.0               63.0                 26.0   

    route_short_nam

In [ ]:
# Select a time-of-day window
t1 = '07:00:00'
t2 = '09:00:00'

cols = ['trip_id', 'start_time', 'direction_id']
f = all_delays.copy().merge(ts[cols])

time_cond = (f['start_time'] >= t1) & (f['start_time'] <= t2)

# Separate directions
charts = []
for direction in [0, 1]:
    dir_cond = f['direction_id'] == direction
    cond = time_cond & dir_cond
    g = f.loc[cond].copy()
    g = g.groupby('route_id').apply(agg_for_box_plot).reset_index()
    g = g.merge(feed.routes)
    g.head()

    # Only keep routes with an average of at least k delay samples per sample trip
    k = 3
    cond = g['num_delays']/g['num_realtime_runs'] >= k
    g = g[cond].copy()

    # Convert to minutes
    box_cols = ['delay_low', 'delay_q1', 'delay_q2', 'delay_q3', 'delay_high']
    g[box_cols] /= 60
    g['route_name'] = g[['route_short_name', 'route_long_name']].apply(
      lambda x: ', '.join(x), axis=1) 

    # Print 10 biggest IQR routes
    print('Routes with largest IQR:')
    print(g.sort_values('iqr', ascending=False).head(5))

    # Plot
    chart = Highchart()
    data = g[['route_name'] + box_cols].copy()
    chart.add_data_set(data.values.tolist(), series_type='boxplot')
    N = data.shape[0]
    date_range = ', '.join(dates)        

    options = {
        'chart': {
            'height': 15*N,
            'inverted': True,
        },
        'plotOptions': {
        },
        'title': {
            'text': 'Tukey box plots of stop delays by route in direction {0}'.format(direction),
        },
        'subtitle': {
            'text': 'Time of day is [{0}, {1}]. <br/>Sample period is {2}'.format(
            t1, t2, date_range)
        },
        'legend': {
            'enabled': False,
        },
        'xAxis': {
            'title': {
                'text': 'Route'
            },
            'type': 'category',
            'labels': {'step': 1},
        },
        'yAxis': {
            'title': {
                'text': 'Minutes'
            },
            'opposite': True,
            'plotLines': [{
                'value': 0,
                'color': 'red',
                'width': 2,
            }]    
        },
        'tooltip': {
    #         'formatter': 'function () {\
    #             return this.point.q3 - this.point.q1 + " min"\
    #         }',
            'valueDecimals': 1,
            'headerFormat': '<b>{point.key}</b><br/>',
            'pointFormat': 'low: {point.low} min<br/>'\
              'Q1: {point.q1} min<br/>'\
              'Q2: {point.median} min<br/>'\
              'Q3: {point.q3} min<br/>'\
              'high: {point.high} min',
        }
    }
    chart.set_dict_options(options)
    charts.append(chart)
    

In [ ]:
charts[0]

In [ ]:
charts[1]

# Tukey box plots of end-stop delays by route

In [35]:
f = all_delays.copy()

# Compute trip end stop sequences
st = feed.stop_times.sort_values(['trip_id', 'stop_sequence'])
st = st.groupby('trip_id').agg('last')['stop_sequence'].reset_index()

# Get delays only for end stops
f = f.merge(st)

f.head(20)

# Aggregate by route
f = f.groupby('route_id').apply(agg_tukey).reset_index()
f = f.merge(feed.routes)

# Only keep routes with an average of at least k end-stop delays per unique trip ID
k = 2
cond = f['num_delays']/f['num_unique_trip_ids'] >= k
g = f[cond].copy()

# Convert to minutes
box_cols = ['delay_low', 'delay_q1', 'delay_q2', 'delay_q3', 'delay_high']
g[box_cols] /= 60
g['route_name'] = g[['route_short_name', 'route_long_name']].apply(
  lambda x: ', '.join(x), axis=1) 

# Print biggest IQR routes
print('Routes with largest IQR:')
print(g.sort_values('iqr', ascending=False).head(5))

# Plot
chart = Highchart()
data = g[['route_name'] + box_cols].copy()
chart.add_data_set(data.values.tolist(), series_type='boxplot')
N = data.shape[0]
date_range = ', '.join(dates)        
    
options = {
    'chart': {
        'height': 15*N,
        'inverted': True,
    },
    'plotOptions': {
    },
    'title': {
        'text': 'Tukey box plots of end-stop delays by route',
    },
    'subtitle': {
        'text': 'Sample period is {0}'.format(date_range)
    },
    'legend': {
        'enabled': False,
    },
    'xAxis': {
        'title': {
            'text': 'Route'
        },
        'type': 'category',
        'labels': {'step': 1},
    },
    'yAxis': {
        'title': {
            'text': 'Minutes'
        },
        'opposite': True,
        'plotLines': [{
            'value': 0,
            'color': 'red',
            'width': 2,
        }]    
    },
    'tooltip': {
#         'formatter': 'function () {\
#             return this.point.q3 - this.point.q1 + " min"\
#         }',
        'valueDecimals': 1,
        'headerFormat': '<b>{point.key}</b><br/>',
        'pointFormat': 'low: {point.low} min<br/>'\
          'Q1: {point.q1} min<br/>'\
          'Q2: {point.median} min<br/>'\
          'Q3: {point.q3} min<br/>'\
          'high: {point.high} min',
    }
}
chart.set_dict_options(options)
chart

Routes with largest IQR:
      route_id  delay_high  delay_low   delay_q1   delay_q2   delay_q3  \
118  route_123   47.300000 -11.416667  -3.554167   3.783333  19.525000   
97   route_102   30.883333 -17.033333  -6.770833   4.358333  13.929167   
135  route_143    7.800000 -14.366667 -13.091667  -5.983333   3.700000   
115  route_120   19.400000 -13.550000  -7.404167  -0.541667   9.337500   
263  route_292   17.183333 -41.883333 -19.704167 -11.500000  -4.616667   

         iqr  num_delays  num_realtime_runs  num_unique_trip_ids  \
118  1384.75        18.0               18.0                  6.0   
97   1242.00        20.0               21.0                  6.0   
135  1007.50         8.0                8.0                  3.0   
115  1004.50        20.0               20.0                  8.0   
263   905.25       582.0              619.0                249.0   

    route_short_name                                    route_long_name  \
118             470X       Papakura To Britoma

# Explore percentage of punctual trips by route

In [ ]:
# Say that a trip is k-punctual if it arrives no more than
# k seconds late at its final stop

k = 5*60

# Get trip final delays
def final_delay(group):
    d = dict()
    d['final_delay'] = group['arrival_delay'].iat[-1]
    return pd.Series(d)

f = st.copy()
f = f.sort_values(['trip_id', 'stop_sequence']).groupby('trip_id').apply(
  final_delay).reset_index()
f = f.merge(ts)

# Calculate fraction of k-punctual trips for each route
def fpt(group):
    d = dict()
    n = group['final_delay'].count()
    cond = group['final_delay'] <= k
    d['frac_punctual_trips'] = group.loc[cond, 'final_delay'].shape[0]/n
    d['num_samples'] = n
    return pd.Series(d)

f = f.groupby('route_id').apply(fpt).reset_index()
f = f.merge(rs)
f['frac_samples'] = f['num_samples']/f['num_trips']
f.sort_values('frac_punctual_trips').head(10)

In [ ]:
# Plot some

# Only keep routes with at least half of trips sampled
cond = f['frac_samples'] >= 0.5
g = f[cond].copy()
g['route_name'] = g[['route_short_name', 'route_long_name']].apply(
  lambda x: ', '.join(x), axis=1) 
g['ppt'] = g['frac_punctual_trips']*100

chart = Highchart()
data = g[['route_name', 'ppt']].copy()
chart.add_data_set(data.values.tolist(), series_type='bar')
N = data.shape[0]
        
options = {
    'chart': {
        'height': 15*N,
    },
    'plotOptions': {
        'series': {
            'minPointLength': 3,
        },
    },
    'title': {
        'text': 'Percentage of 5-minute-punctual trips by route',
    },
    'subtitle': {
        'text': 'Sample period is {0}'.format(date)
    },
    'legend': {
        'enabled': False,
    },
    'xAxis': {
        'title': {
            'text': 'Route'
        },
        'type': 'category',
        'labels': {'step': 1},
    },
    'yAxis': {
        'title': {
            'text': 'Percentage'
        },
        'opposite': True,
        'max': 100,
    },
    'tooltip': {
        'headerFormat': '<b>{point.key}</b><br/>',
        'pointFormat': '{point.y}%',
        'valueDecimals': 0,
    }
}
chart.set_dict_options(options)
chart

In [ ]:
# # Fill departure delays.

# # 1. First pass. 
# # Fill actual departure times via distance interpolation.
# # Then use these to fill departure delays.
# f = st.copy()

# print('num trips=', f['trip_id'].nunique())

# f['actual_departure_time'] = f['departure_time'].map(gt.timestr_to_seconds) +\
#   f['departure_delay']

# def fill_dtimes(group):
#     indices = np.where(group['actual_departure_time'].notnull())[0]
#     if not indices.size:
#         return group
#     times = group['actual_departure_time'].ix[indices]
#     dists = group['shape_dist_traveled'].ix[indices]
#     ds = group['shape_dist_traveled']
#     ts = np.interp(ds, dists, times)
#     group['actual_departure_time'] = ts
#     return group

# f = f.groupby('trip_id').apply(fill_dtimes)

# # Round departure times
# f['actual_departure_time'] = f['actual_departure_time'].round()

# # Compute delays
# f['departure_delay'] = f['actual_departure_time'] -\
#   f['arrival_time'].map(gt.timestr_to_seconds)
    

# # 2. Second pass. 
# # For the remaining null departure delays, fill them by forward fill
# # and then backward fill.
# f['departure_delay'] = f['departure_delay'].fillna(method='ffill').fillna(method='bfill')     

# f['actual_departure_time'] = f['departure_time'].map(gt.timestr_to_seconds) +\
#   f['departure_delay']

# # Convert back to time strings
# f['actual_departure_time'] = f['actual_departure_time'].map(
#   lambda x: gt.timestr_to_seconds(x, inverse=True))

# f.T